In [1]:
from pricing_library.market import Market
from pricing_library.option import Option
from pricing_library.trinomial_tree import TrinomialTree
from pricing_library.greeks import delta
from datetime import datetime


opt = Option(
    option_type="call",
    exercise_type="eu",
    strike_price=101,
    maturity_date=datetime(2024, 7, 1),
)
mkt = Market(
    interest_rate=0.02,
    volatility=0.25,
    spot_price=100,
    dividend_price=0,
    dividend_ex_date=datetime(2024, 3, 1),
)
pricing_date = datetime(2023, 9, 1)

In [2]:
delta(mkt, 100, pricing_date, opt, 0.01)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
t = TrinomialTree(
    market=mkt,
    pricing_date=pricing_date,
    n_steps=100,
)
price = t.price(
    opt=opt,
    draw_tree=False,
)
# discount factor

In [ ]:
price

In [ ]:
t.alpha, t.delta_t, t.root.p_down, t.root.p_mid, t.root.p_up

In [ ]:
# params = {"n": 2000,
#           "s0": 100,
#           "vol": 0.25,
#           "rate": 0.04,
#           "strike": 102,
#           "maturity": datetime.date(year=2024, month=9, day=19),
#           "t0": datetime.date(year=2023, month=9, day=20),
#           "market": "european",
#           "div": 0
#           }